# User Analysis

In here we are analysing the data in `data/users/`. Before pre-processing, **we have 341,018 users** that appear in our games network found in `data/games/network_raw`. 

In [7]:
import pandas as pd

user_prefs_filename = "../data/users/user_preferences_with_metadata.csv"
user_prefs_df = pd.read_csv(user_prefs_filename)
user_prefs_df.describe(include='all', datetime_is_numeric=True)

,user,signup_date,location,num_games,games
count,341018,341018,341018,341018.000000,341018
unique,341018,340454,258,NaN,89185
top,j5wzz2qj,Null,us,NaN,k6q4rqzd
freq,1,74,101936,NaN,5296
mean,NaN,NaN,NaN,1.980444,NaN
std,NaN,NaN,NaN,7.359630,NaN
min,NaN,NaN,NaN,1.000000,NaN
25%,NaN,NaN,NaN,1.000000,NaN
50%,NaN,NaN,NaN,1.000000,NaN
75%,NaN,NaN,NaN,2.000000,NaN


In [9]:
user_prefs_df = user_prefs_df[(user_prefs_df['signup_date'].notna()) & (user_prefs_df['signup_date'] != "Null")]
user_prefs_df['signup_date'] = pd.to_datetime(
    user_prefs_df['signup_date'], format='%Y-%m-%dT%H:%M:%SZ')
user_prefs_df['signup_date'] = pd.to_datetime(
    user_prefs_df['signup_date'].dt.strftime('%Y-%m-%d'))
user_prefs_df.describe(include='all', datetime_is_numeric=True)

,user,signup_date,location,num_games,games
count,340944,340944,340944,340944.000000,340944
unique,340944,NaN,257,NaN,89150
top,j5wzz2qj,NaN,us,NaN,k6q4rqzd
freq,1,NaN,101936,NaN,5295
mean,NaN,2020-07-14 00:02:53.842038528,NaN,1.979486,NaN
min,NaN,2014-01-06 00:00:00,NaN,1.000000,NaN
25%,NaN,2019-10-10 00:00:00,NaN,1.000000,NaN
50%,NaN,2021-01-09 00:00:00,NaN,1.000000,NaN
75%,NaN,2021-10-17 00:00:00,NaN,2.000000,NaN
max,NaN,2023-02-18 00:00:00,NaN,2059.000000,NaN


After processing our dataframe so that we remove users that have invalid signup dates, **we now have 320,944 users**. **During this process we removed 74 users**. In the collection, if a user could not found by the speedrun.com API, we set their signup date as "Null". This is explained by users deleting their account, but their runs still being avaialble on the leaderboards.

In [89]:
grouped_df = user_prefs_df.groupby(['location'])
grouped_sum_df = grouped_df.sum(numeric_only=True)
grouped_sum_df['number'] = grouped_df.count().iloc[:, 0]
grouped_sum_df.reset_index(inplace=True)
grouped_sum_df['location'] = grouped_sum_df['location'].str.upper()

In [122]:
country_df = pd.read_csv("../data/users/countries_codes_and_coordinates.csv")
country_df = country_df[['Country', 'Alpha-2 code']].rename(columns={'Alpha-2 code': 'location', 'Country': 'country'})
country_df['location'] = country_df['location'].str.strip()
country_df['location'] = country_df['location'].str.replace('\"', '', -1)

In [128]:
merged_df = country_df.merge(grouped_sum_df, on='location', how='right')
merged_df.sort_values(by='number').tail(15).iloc[::-1]

,country,location,num_games,number
246,United States,US,232240,101936
0,NaN,NONE,79092,61363
40,Canada,CA,35439,15046
87,NaN,GB/ENG,25309,12948
84,France,FR,25898,12528
61,Germany,DE,24398,11901
13,Australia,AU,18762,9841
33,Brazil,BR,18724,9089
204,Russian Federation,RU,15366,7655
205,Russia,RU,15366,7655


In [131]:
merged_df.sort_values(by='number', ascending=True).head(15)

,country,location,num_games,number
107,Heard Island and McDonald Islands,HM,2,1
32,NaN,BQ,1,1
64,Dominica,DM,1,1
212,"Saint Helena, Ascension and Tristan da Cunha",SH,7,1
226,Turks and Caicos Islands,TC,1,1
153,NaN,MF,1,1
24,Burundi,BI,9,1
47,Côte d'Ivoire,CI,1,1
48,Ivory Coast,CI,1,1
143,Lesotho,LS,1,1
